# Imports

In [29]:
from collections import defaultdict
from dataclasses import dataclass
from enum import Enum
from queue import PriorityQueue

import numpy as np
import pandas as pd
import simpy

# Yield

Simpy uses **yield** for creating processes.

**yield** returns the next element of a generator.

In [12]:
def create_generator():
    for i in range(3):
        yield i


for i in create_generator():
    print(i)

0
1
2


Combining yields

In [13]:
def create_generator_outer():
    for i in range(3):
        yield i, create_generator()


for i, g in create_generator_outer():
    for j in g:
        print('%i %i' % (i, j))

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2


# Basic examples

## Timeout event

`env.timeout(delay)` adds a timeout event to the scheduler that happens after the delay.

In [18]:
counter = defaultdict(int)


def clock(env, name, rate):
    while True:
        counter[name] += 1
        yield env.timeout(np.random.geometric(rate))
        print(name)


env = simpy.Environment()
env.process(clock(env, 'fast', 0.5))
env.process(clock(env, 'slow', 0.2))
env.run(until=10)
print(dict(counter))

slow
fast
fast
slow
fast
{'fast': 4, 'slow': 3}


## Processes

- A process defines a sequence of events.
- A process is an event => we can wait for processes to finish.

In [19]:
def process_1(env):
    while True:
        print('t_%d: Hi from P1' % env.now)
        yield env.timeout(3)
        yield env.process(process_2(env))


def process_2(env):
    while True:
        print('t_%d: Hi from P2' % env.now)
        yield env.timeout(1)
        yield env.process(process_1(env))


env = simpy.Environment()
env.process(process_2(env))
env.run(until=10)

t_0: Hi from P2
t_1: Hi from P1
t_4: Hi from P2
t_5: Hi from P1
t_8: Hi from P2
t_9: Hi from P1


## Queue

In [20]:
queue = []
idle = True
customer_number = 0

avg_inter_arrival_time = 5
avg_service_duration = 7

waiting_times = []


class Customer:
    next_id = 0

    def __init__(self, arrival_time):
        self.name = 'C_%d' % Customer.next_id
        Customer.next_id += 1
        self.arrival_time = arrival_time

    def departure(self, departure_time):
        self.departure_time = departure_time
        waiting_times.append(departure_time - self.arrival_time)

    def __str__(self):
        return self.name


def serve():
    global idle

    idle = False
    service_duration = np.random.geometric(1.0 / avg_service_duration)
    print('%d: serve %s for %s periods' % (env.now, queue[0], service_duration))
    yield env.timeout(service_duration)

    # Customer departure
    customer = queue.pop(0)
    customer.departure(env.now)
    print('%d: departure of %s, queue length is %d' % (env.now, customer, len(queue)))

    # Continue working or sleep
    if len(queue) == 0:
        idle = True  # queue empty - going to sleep
    else:
        yield env.process(serve())  # serve another customer


def arrival():
    while True:
        customer = Customer(env.now)
        queue.append(customer)
        print('%d: arrival of %s, queue length is %d' % (env.now, customer, len(queue)))

        if idle:
            env.process(serve())

        inter_arrival_time = np.random.geometric(1.0 / avg_inter_arrival_time)
        yield env.timeout(inter_arrival_time)


env = simpy.Environment()
env.process(arrival())
env.run(until=50)
print()
print('Final queue length is %d' % len(queue))
print(pd.Series(waiting_times).describe())

0: arrival of C_0, queue length is 1
0: serve C_0 for 7 periods
7: departure of C_0, queue length is 0
12: arrival of C_1, queue length is 1
12: serve C_1 for 3 periods
15: departure of C_1, queue length is 0
25: arrival of C_2, queue length is 1
25: serve C_2 for 9 periods
28: arrival of C_3, queue length is 2
31: arrival of C_4, queue length is 3
34: departure of C_2, queue length is 2
34: serve C_3 for 5 periods
37: arrival of C_5, queue length is 3
39: departure of C_3, queue length is 2
39: serve C_4 for 1 periods
40: departure of C_4, queue length is 1
40: serve C_5 for 5 periods
45: departure of C_5, queue length is 0
47: arrival of C_6, queue length is 1
47: serve C_6 for 2 periods
49: arrival of C_7, queue length is 2
49: departure of C_6, queue length is 1
49: serve C_7 for 9 periods

Final queue length is 1
count     7.000000
mean      7.000000
std       3.316625
min       2.000000
25%       5.000000
50%       8.000000
75%       9.000000
max      11.000000
dtype: float64


# Events

## Callback

A callback is attached to an event and triggered after the event.

In [25]:
env = simpy.Environment()


def my_callback(event):
    print('Callback')
    print(event)


def my_process():
    event = env.event()
    event.callbacks.append(my_callback)
    yield event.succeed()
    print('Done')


env.process(my_process())
env.run(until=10)

Callback
<Event() object at 0x117872c10>
Done


## Example usages for event

In [26]:
env = simpy.Environment()


class School:
    def __init__(self):
        self.class_ends = env.event()
        self.bell_proc = env.process(self.bell())
        self.pupil_procs = [env.process(self.pupil()) for i in range(3)]

    def bell(self):
        for i in range(2):
            yield env.timeout(45)
            self.class_ends.succeed()  # end class
            print('%d: Class ended' % env.now)
            self.class_ends = env.event()

    def pupil(self):
        while True:
            yield self.class_ends  # this event is shared!
            print(r' \o/')


school = School()
env.run(until=1000)

45: Class ended
 \o/
 \o/
 \o/
90: Class ended
 \o/
 \o/
 \o/


## Processes are events

In [ ]:
env = simpy.Environment()


def sub():
    yield env.timeout(1)
    return 23


def parent():
    ret = yield env.process(sub())
    return ret


env.run(env.process(parent()))

## Waiting for multiple events

In [ ]:
env = simpy.Environment()


def test_or():
    t1, t2 = env.timeout(1, value='spam'), env.timeout(2, value='eggs')
    ret = yield t1 | t2

    assert ret == {t1: 'spam'}  # spam is triggered first
    assert env.now == 1


def test_and():
    t1, t2 = env.timeout(1, value='spam'), env.timeout(2, value='eggs')
    ret = yield t1 & t2

    assert ret == {t1: 'spam', t2: 'eggs'}
    assert env.now == 2


env.process(test_or())
env.process(test_and())
env.run()

**Events at the same time**

In [ ]:
env = simpy.Environment()


def test_or():
    t1, t2 = env.timeout(1, value='spam'), env.timeout(1, value='eggs')
    ret = yield t1 | t2
    print(len(list(ret.items())))
    print(ret)


proc = env.process(test_or())
env.run()

## Wait for a timeout OR until a trigger is activated

In [ ]:
env = simpy.Environment()


class Team:

    def repair(self):
        repaired = env.timeout(10)
        self.interrupted = env.event()  # used to trigger interruption
        ret = yield self.interrupted | repaired

        if repaired in ret:
            print('repaired')
        if self.interrupted in ret:
            print('interrupted')


def interrupt(team):
    yield env.timeout(10)
    team.interrupted.succeed()  # pull trigger


team = Team()
env.process(team.repair())
env.process(interrupt(team))
env.run()

Careful when the events occur at the exact same time!

- It may be, that both events are in ret or only one. 
- It depands also on the order of scheduling processes.

=> Process interruption is more predictable, as interrupts are prioritized!

## Events at the same time

Interrupts and process initialization events are priorized over other events, if they occur at the same time.

If the priorititzation is the same, the one that was added first to the scheduler is executed first.

In [ ]:
env = simpy.Environment()


class A:

    def __init__(self):
        env.process(self.proc1())
        env.process(self.proc2())

    def proc1(self):
        yield env.timeout(10)
        print('1st')
        env.process(self.another_proc())

    def proc2(self):
        yield env.timeout(10)
        print('3rd')

    def another_proc(self):
        print('2nd')
        yield env.timeout(1)


a = A()
env.run()

## Event fail

TODO

In [ ]:



env = simpy.Environment()


class A:

    def __init__(self):
        env.process(self.proc2())
        env.process(self.proc1())

    def proc1(self):
        self.to = env.timeout(10)
        self.trigger = env.event()
        ret = yield self.to | self.trigger
        print(ret)

        if self.to in ret:
            print('TO')

        if self.trigger in ret:
            print('TR')

    def proc2(self):
        yield env.timeout(9)
        self.trigger.succeed()


a = A()
env.run()

# Process Interaction

## Sleep until woken up

In [ ]:
env = simpy.Environment()
my_trigger = env.event()


def my_process():
    print('Start process')
    yield my_trigger
    print('Triggered at %d' % env.now)


def pull_trigger():
    yield env.timeout(17)
    print('Pull trigger')
    my_trigger.succeed()


env.process(my_process())
env.process(pull_trigger())
env.run(until=100)


In [ ]:
# Sleep until woken up OR sth. else happens

env = simpy.Environment()
my_trigger = env.event()


def delay_process_1():
    yield env.timeout(11)
    print('Delay 1')


def delay_process_2():
    yield env.timeout(17)
    print('Delay 2')


def my_process():
    print('Start process')
    delay = env.process(delay_process())
    trigger = env.process(trigger_process())
    yield trigger | delay
    if delay.triggered:
        print('Random_delay at %d' % env.now)
        trigger.interrupt()
    if trigger.triggered:
        print('Trigger pulled at %d' % env.now)
        #random_delay.interrupt()
    print('Done at %d' % env.now)


env.process(my_process())
trigger_process = env.process(pull_trigger())
env.run(until=100)

## Interrupt a process

We wait for one of two events:
 
+ parking is finished, or
+ charging is finished.  

If parking is finished before charging, the charging process is interrupted. 


In [ ]:
class EV:
    """ An electric vehicle """

    def __init__(self, env):
        self.env = env
        self.drive_proc = env.process(self.drive())

    def drive(self):
        """ Alternate between: drive for a while, then park and charge. """
        while True:
            # Drive
            yield self.env.timeout(20)

            # Park 
            print('%d: start parking at' % self.env.now)
            charging = self.env.process(self.charge_battery())
            parking = self.env.timeout(60)
            yield charging | parking

            if not charging.triggered:
                charging.interrupt('need to go!')

            print('%d: stop parking' % self.env.now)

    def charge_battery(self):
        print('%d: charging started' % self.env.now)
        try:
            yield self.env.timeout(70)  # takes longer than parking
            print('%d: battery charged' % self.env.now)
        except simpy.Interrupt as i:
            # Got interrupted before the charging was done.
            print('%d: charging interrupted, cause: %s' % (self.env.now, i.cause))


env = simpy.Environment()
ev = EV(env)
env.run(until=100)

# Rolling planning example

There is a list of tasks. 
A plan is an ordered sequence of the tasks. 
We want to update the plan some time. 
However, we do not want to interrupt tasks. 
Therefore, we schedule tasks one-by-one. 


In [49]:
@dataclass
class Task:
    id: int
    duration: float


plan = [Task(id=i, duration=10) for i in range(4)]


def execute_task(env, task):
    print(f"{env.now:>4}: Execute task {task.id}")
    yield env.timeout(task.duration)
    print(f"{env.now:>4}: Completed task {task.id}")


def execute_plan(env, plan):
    while len(plan):
        t = plan.pop(0)
        yield env.process(execute_task(env, t))


def change_plan(env, plan, delay):
    yield env.timeout(delay)
    plan.reverse()


env = simpy.Environment()
env.process(execute_plan(env, plan))
env.process(change_plan(env, plan, 5))
env.run(until=100)

   0: Execute task 0
  10: Completed task 0
  10: Execute task 3
  20: Completed task 3
  20: Execute task 2
  30: Completed task 2
  30: Execute task 1
  40: Completed task 1


# Turbine example

## Helper

In [ ]:
def print_q(q):
    lst = [i for i in q.queue]
    lst.sort()
    print(lst)

## Generate maintenance incidents

- List with unscheduled incidents
- List with scheduled services 
- Leave a gap of at least 3 periods corresponding to the repair time

In [ ]:
def generate_incidents(prob, gap):
    ret = []
    i = 0
    while i < 100:
        if np.random.rand() < prob:
            ret.append(i)
            i += gap
        else:
            i += 1
    return ret


def generate_all():
    # Unscheduled
    global incidents
    incidents = generate_incidents(0.1, 4)
    print(incidents)

    # Scheduled
    global incidents_scheduled
    incidents_scheduled = generate_incidents(0.05, 10)
    print(incidents_scheduled)
    global n_jobs
    n_jobs = len(incidents) + len(incidents_scheduled)
    print('# jobs =', n_jobs)

## Setup simulation

In [ ]:
def setup_sim():
    incident_q = PriorityQueue()
    for i in incidents:
        incident_q.put(i)
    print_q(incident_q)

    incidents_scheduled_q = incidents_scheduled.copy()
    print(incidents_scheduled_q)

    global env
    env = simpy.Environment()
    env.incidents = incident_q
    env.incidents_scheduled = incidents_scheduled_q
    env.until = 200
    env.threshold = 4
    print('Threshold =', env.threshold)

    env.turbine = Turbine(env.incidents, env.incidents_scheduled)

## Job

In [ ]:
class JobType(Enum):
    SCHEDULED = 1
    UNSCHEDULED = 2
    MERGED = 3


class Job():
    next_id = 0

    def __init__(self, job_type=JobType.UNSCHEDULED, next_process=None, info=''):
        self.id = Job.next_id
        Job.next_id += 1
        self.t = env.now
        self.job_type = job_type
        self.next_process = next_process
        self.info = info

    def __str__(self):
        return 'Job [id={}, t={}, job_type={}, next_process={}, info={}]'.format(
            self.id, self.t, self.job_type, self.next_process, self.info)


## Turbine

In [ ]:
class Turbine():

    def __init__(self, incidents, incidents_scheduled):
        """
        incidents: PriorityQueue
        incidents_scheduled: list (sorted)
        """
        self.incidents = incidents
        self.incidents_scheduled = incidents_scheduled

        # State
        self.stop_switch = False
        self.completed_scheduled_services = []
        self.initiated_scheduled_services = []
        self.completed_unscheduled_failures = []
        self.initiated_unscheduled_failures = []

        # Collector
        self.job_log = []

        # Processes
        env.process(self.fail_process())
        env.process(self.scheduled_maintenance_process())

    def include(self, job):
        if len(self.incidents_scheduled) == 0:  # no outstanding scheduled services left
            return job

        h = self.incidents_scheduled[0]
        if h - env.now > threshold:
            return job

        return Job(job_type=JobType.MERGED, next_process='fail', info='%s+%s' % (job.t, h))

    def fail_process(self):
        #print('%d: Fail process' % (env.now))
        next_dt = self.get_next_failure()
        #print('%d: Next dt = %s' % (env.now, next_dt))
        yield env.timeout(next_dt - env.now)

        # Maintenance necessary
        log = '%d: Failure' % env.now
        job = Job(job_type=JobType.UNSCHEDULED, next_process='fail')
        new_job = self.include(job)

        if new_job == job:
            log += ' - Not merged %s' % (job)
            print(log)
            env.process(self.repair_process(job))
        else:
            log += ' - Merged %s to %s' % (job, new_job)
            print(log)
            self.stop_switch = True
            self.incidents_scheduled.pop(0)  # rm from list
            self.initiated_scheduled_services.append(new_job.t)
            env.process(self.repair_process(new_job))

    def get_next_failure(self):
        #print('%d: Get next failure' % (env.now))
        next_dt = -1
        while next_dt < env.now and self.incidents.qsize() > 0:
            next_dt = self.incidents.get()
        if next_dt < env.now:
            return until + 1  # never
        return next_dt

    def repair_process(self, job):
        #print('%d: Repair process, %s' % (env.now, job))
        if job.job_type == JobType.MERGED:
            yield env.timeout(4)
        else:
            yield env.timeout(3)

        print('%d: Repaired, %s' % (env.now, job))
        if job.job_type == JobType.SCHEDULED:
            self.completed_scheduled_services.append(job)
        if job.job_type == JobType.UNSCHEDULED:
            self.completed_unscheduled_failures.append(job)

        self.job_log.append(job)

        # Start next process
        if job.next_process is None:
            return

        if job.next_process == 'fail':
            env.process(self.fail_process())
            return

        if job.next_process == 'scheduled':
            # no need to start new process
            return

    def scheduled_maintenance_process(self):
        #print('%d: Scheduled maintenance process' % (env.now))

        if len(self.incidents_scheduled) == 0:
            return  # terminate scheduled service process for good

        next_scheduled_service = self.incidents_scheduled[0]
        yield env.timeout(next_scheduled_service - env.now)

        log = '%d: Time for scheduled maintenance' % (env.now)

        if self.stop_switch:
            log += ' - Already initiated'
            print(log)
            self.stop_switch = False
            # already removed from list
            # already added to initiated list
            env.process(self.scheduled_maintenance_process())
            return

        self.incidents_scheduled.pop(0)  # Remove from the list
        self.initiated_scheduled_services.append(next_scheduled_service)  # Add to initiated list

        # Create and add job
        job = Job(job_type=JobType.SCHEDULED)
        log += ' - Initiated %s' % job
        print(log)
        env.process(self.repair_process(job))
        env.process(self.scheduled_maintenance_process())

    def __str__(self):
        return self.name



## Check results

In [ ]:
def check_result(env):
    counter = 0
    for job in env.turbine.job_log:
        print(job)
        counter += 1 if job.job_type != JobType.MERGED else 2
    print('Counter =', counter, ', # jobs =', n_jobs)
    assert counter == n_jobs

## Run

In [ ]:
for i in range(20):
    generate_all()
    setup_sim()
    env.run(until=env.until)
    check_result(env)

## Single run

In [ ]:
setup_sim()
env.run(until=env.until)
check_result(env)

# Shared resource


In [ ]:
import threading
import simpy

env = simpy.Environment()


class TechPool:

    def __init__(self, tech_count):
        self.tech_count = tech_count

        # State
        self.available_tech_count = tech_count
        self.lock = threading.Lock()

    def get_techs(self, n, who='?'):
        self.lock.acquire()
        if n > self.available_tech_count:
            raise Exception('Not enough techs')
        self.available_tech_count -= n
        print(env.now, 'TechPool - %d for %s = %d' % (n, who, self.available_tech_count))
        self.lock.release()
        return n


def process_1(pool):
    print('Start process 1')
    yield env.timeout(1)
    print('Get 1')
    pool.get_techs(1)


def process_2(pool):
    print('Start process 2')
    yield env.timeout(1)
    print('Get 2')
    pool.get_techs(2)


pool = TechPool(3)
env.process(process_1(pool))
env.process(process_2(pool))
env.run(until=100)
